<a href="https://colab.research.google.com/github/piiop/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [40]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [41]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [42]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [43]:
# Create a list of at least 10 column names to use as X data
column_list = ['Age','EnvironmentSatisfaction', 'HourlyRate',
               'JobInvolvement','JobLevel','JobSatisfaction',
               'NumCompaniesWorked','YearsAtCompany','YearsInCurrentRole',
               'YearsWithCurrManager']


# Create X_df using your selected columns
X_df = attrition_df[column_list]

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
EnvironmentSatisfaction,int64
HourlyRate,int64
JobInvolvement,int64
JobLevel,int64
JobSatisfaction,int64
NumCompaniesWorked,int64
YearsAtCompany,int64
YearsInCurrentRole,int64
YearsWithCurrManager,int64


In [44]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=199)


In [45]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# All training data already int64

In [46]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)


# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [47]:
# Create a OneHotEncoder for the Department column

dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoder.fit(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
dept_names = dept_encoder.get_feature_names_out(['Department'])
dept_train_encoded = pd.DataFrame(dept_encoder.transform(y_train['Department'].values.reshape(-1,1)), columns=dept_names)
dept_test_encoded = pd.DataFrame(dept_encoder.transform(y_test['Department'].values.reshape(-1,1)), columns=dept_names)



In [48]:

# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attr_encoder.fit(y_train['Attrition'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
attr_names = attr_encoder.get_feature_names_out(['Attrition'])
attr_train_encoded = pd.DataFrame(attr_encoder.transform(y_train['Attrition'].values.reshape(-1,1)), columns=attr_names)
attr_test_encoded = pd.DataFrame(attr_encoder.transform(y_test['Attrition'].values.reshape(-1,1)), columns=attr_names)

## Create, Compile, and Train the Model

In [49]:
# Find the number of columns in the X training data
columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(columns,))

# Create at least two shared layers
shared1 = layers.Dense(units=64, activation='relu')(input_layer)
shared2 = layers.Dense(units=32, activation='relu')(shared1)


In [50]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=32, activation='relu')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax',name='department_output')(department_hidden)


In [51]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=32, activation='relu')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid',name='attrition_output')(attrition_hidden)


In [52]:
# Create the model
model = Model(inputs=input_layer,
              outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output' : 'categorical_crossentropy',
                    'attrition_output' : 'binary_crossentropy'},
              metrics={'department_output' : 'accuracy',
                    'attrition_output' : 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 64)             │            704 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 32)             │          2,080 │ dense_14[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 32)             │          1,056 │ dense_15[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 32)             │          1,056 │ dense_15[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dense_17[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,061 (19.77 KB)

 Trainable params: 5,061 (19.77 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
# Train the model
model.fit(X_train_scaled,
        {
        'department_output': dept_train_encoded,
        'attrition_output': attr_train_encoded
        },
          epochs=100,
          batch_size=32,
          )


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - attrition_output_accuracy: 0.8273 - department_output_accuracy: 0.6355 - loss: 1.4931
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8426 - department_output_accuracy: 0.6129 - loss: 1.2489
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8385 - department_output_accuracy: 0.6406 - loss: 1.1778
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8390 - department_output_accuracy: 0.6386 - loss: 1.1803
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8322 - department_output_accuracy: 0.6271 - loss: 1.1770
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8458 - department_output_accuracy: 0.6640 - loss: 1.1158
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8481 - department_output_accuracy: 0.6468 - loss: 1.1200
Epoch 8/100
35/35 ━━━━━━━━━

In [59]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, {
        'department_output': dept_test_encoded,
        'attrition_output': attr_test_encoded
    })

display(results)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8083 - department_output_accuracy: 0.5883 - loss: 1.9650 


[1.7931773662567139, 0.8179348111152649, 0.60597825050354]

In [60]:
# Print the accuracy for both department and attrition
department_accuracy = results[1]
attrition_accuracy = results[2]

# Print the results
print(f"Department predictions accuracy: {department_accuracy:.4f}")
print(f"Attrition predictions accuracy: {attrition_accuracy:.4f}")

Department predictions accuracy: 0.8179
Attrition predictions accuracy: 0.6060


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.